# COSMOS2020 Quiescent Galaxy Classifier

### Needed Libraries

In [10]:
import pandas as pd
import joblib
import warnings
warnings.filterwarnings('ignore')

### Needed Function

In [11]:
def calculate_pairwise_differences(data):
    """
    Calculate all pairwise differences between columns in a DataFrame (using for deriving colors from bands).
    
    For each unique pair of columns (i,j) where i < j, creates a new column
    named by concatenating the two column names, containing the element-wise
    difference between column i and column j.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input DataFrame containing numerical columns to compare
        
    Returns:
    --------
    pandas.DataFrame
        Original DataFrame with new columns added for each pairwise difference
    """
    
    ln = len(data.columns)  # Get number of columns in DataFrame
    
    # Iterate through all possible column pairs
    for i in range(ln):
        for j in range(ln):
            if i < j:  # Ensure we only do unique pairs (i < j)
                # Create new column name by concatenating the two column names
                diff_name = f"{data.columns[i]}{data.columns[j]}"
                
                # Calculate and store the difference between columns i and j
                data[diff_name] = data[data.columns[i]] - data[data.columns[j]]
    
    return data

### Loading Models

In [12]:
model = joblib.load('ML_Model_Classifier.pkl') # Load model classifier
scaler = joblib.load('ML_Model_Scaler.pkl') # Load scaler fitted on training data

### COSMOS2020 Data (0.2 < redshift < 3.5)

In [13]:
df = pd.read_csv('COSMOS2020_ml_classifications.csv.gz') # Load COSMOS2020 data

selected_df = df[['cfht_u_mag', 'acs_f814w_mag', 'uvista_y_mag', 'uvista_j_mag', 'uvista_h_mag', 
                  'uvista_ks_mag', 'irac_ch1_mag', 'irac_ch2_mag']] # select 8 bands (Warning: Order is important)

In [14]:
X_new = calculate_pairwise_differences(selected_df).iloc[:,8:] # Construct 28 colors form 8 selected bands

X_new_scaled = scaler.transform(X_new.values) # # Scale the new galaxies using the previously fitted scaler (training data)

y_new = model.predict(X_new_scaled) # Predict galaxy types: True for Quiescent, False for Star-forming

In [15]:
n_qg = sum(y_new) # Count quiescent galaxies

n_sfg = len(y_new) - n_qg # Count star-forming galaxies

print(f"The number of quiescent galaxies: {n_qg} \nThe number of star-forming galaxies: {n_sfg}")

The number of quiescent galaxies: 36950 
The number of star-forming galaxies: 328927
